# DSCI 525: Milestone 1 Group 20

## Group Members
- Lauren Zung
- Xinru Lu
- Spencer Gerlach

# Part 1 & 2: Contract & Repo

- Completed by Lauren Zung

# Part 3: Downloading the Data

- Spencer Gerlach

In [1]:
# Imports

import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

- Change directory to location files stored.

- Assuming we can't save data to our repo.

> Will need to be updated depending on who is running the notebook

In [2]:
%cd /Users/spencergerlach/Desktop/figshare

/Users/spencergerlach/Desktop/figshare


In [3]:
# Complete metadata required for API request

article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshare-nswrain" # update depending on user

In [4]:
# GET request

response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]

- Now, download the file `data.zip`

In [8]:
%%time
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + "/" + file["name"])

CPU times: user 3.13 s, sys: 4.34 s, total: 7.46 s
Wall time: 1min 12s


In [9]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), "r") as f:
    f.extractall(output_directory)

CPU times: user 18.7 s, sys: 4.95 s, total: 23.7 s
Wall time: 29.5 s


In [11]:
# check all the file names
%ls -ltr /Users/spencergerlach/Desktop/figshare/figshare-nswrain/

total 12320624
-rw-r--r--   1 spencergerlach  staff  814041183 Mar 28 18:11 data.zip
-rw-r--r--   1 spencergerlach  staff   95376895 Mar 28 18:11 MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
-rw-r--r--   1 spencergerlach  staff   94960113 Mar 28 18:11 AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 spencergerlach  staff   82474546 Mar 28 18:11 NorESM2-LM_daily_rainfall_NSW.csv
-rw-r--r--   1 spencergerlach  staff  127613760 Mar 28 18:11 ACCESS-CM2_daily_rainfall_NSW.csv
-rw-r--r--   1 spencergerlach  staff  232118894 Mar 28 18:11 FGOALS-f3-L_daily_rainfall_NSW.csv
-rw-r--r--   1 spencergerlach  staff  330360682 Mar 28 18:11 CMCC-CM2-HR4_daily_rainfall_NSW.csv
-rw-r--r--   1 spencergerlach  staff  254009247 Mar 28 18:11 MRI-ESM2-0_daily_rainfall_NSW.csv
-rw-r--r--   1 spencergerlach  staff  235661418 Mar 28 18:11 GFDL-CM4_daily_rainfall_NSW.csv
-rw-r--r--   1 spencergerlach  staff  294260911 Mar 28 18:11 BCC-CSM2-MR_daily_rainfall_NSW.csv
-rw-r--r--   1 spencergerlach  staff  295768615 M

# Part 4: Combine the Files with Python

- Spencer Gerlach

In [12]:
df_test = pd.read_csv("/Users/spencergerlach/Desktop/figshare/figshare-nswrain/AWI-ESM-1-1-LR_daily_rainfall_NSW.csv")
df_test.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,3.129635e-02
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,1.083881e-13
2,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,1.056313e-13
3,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,1.080510e-13
4,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,9.914916e-14


In [15]:
df_test2 = pd.read_csv("/Users/spencergerlach/Desktop/figshare/figshare-nswrain/ACCESS-CM2_daily_rainfall_NSW.csv")
df_test2.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13
1,1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00
2,1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00
3,1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00
4,1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02


- From these results, we can now proceed with reading and combining all CSVs (except `observed_daily_rainfall_SYD.csv`).

- Use columns from the test CSVs above

In [19]:
%%time
# Combine into one CSV
files = glob.glob('/Users/spencergerlach/Desktop/figshare/figshare-nswrain/*.csv') 
# Manually removed observed_daily_rainfall_SYD.csv from the data folder
df = pd.concat((pd.read_csv(file).assign(model=re.findall("/([^_]*)", file)[0]) for file in files))
df.to_csv("/Users/spencergerlach/Desktop/figshare/figshare-nswrain/combined_data.csv") # Use absolute path for now

CPU times: user 12min 38s, sys: 1min 14s, total: 13min 52s
Wall time: 16min 5s


### Part 4: Time Taken to Combine CSV files

| Team Member | Operating System | RAM | Processor | Is SSD | Time Taken |
|-------------|------------------|-----|-----------|--------|------------|
|  Spencer    |   MacOS 12.6     |  8  | intel i5  |   Yes  |  16m 5s    |
|             |                  |     |           |        |            |
|             |                  |     |           |        |            |
|             |                  |     |           |        |            |
